In [1]:
import pandas as pd
import glob
import numpy as np
import copy
import matplotlib.pyplot as plt
%matplotlib inline
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from pathlib import Path
import cv2
from time import time
from threading import Thread
from queue import Queue

In [2]:
time_col = 'TIME'
position_cols = ['BPOGX', 'BPOGY']
reading_col = 'FPOGID'
fixation_col = 'Fixation'
fixation_id_col = 'Fixation_ID'
velocity_col = 'Velocity'
distances_col = 'Distances'
periods_col = 'Periods'
# new pos cols
abs_position_cols = ['ABS_X', 'ABS_Y']


In [3]:
pixel_width = 1920
pixel_height = 1080
params = [pixel_width, pixel_height]
margin = 100

green = (0, 255, 0)
red = (0, 0, 255)
yellow = (0,255,255)
blue = (255,0,0)

In [4]:
dfs = {}
for file in glob.glob("data/*/*text2*.csv"):
    df = pd.read_csv(file)
    df_cols = df.columns.values
    df_cols[4] = 'TIME'
    df.columns = df_cols
    dfs[file] = df[position_cols + [reading_col] + [time_col]]

<h3>Convert relative coors to absolute

In [5]:
for filename, df in dfs.items():
    for a, r, p in zip(abs_position_cols, position_cols, params):
        df[a] = df[r]*p

<h3>Calculate distances between points and velocity

In [6]:
diff_cols = ['DIFF_'+c for c in abs_position_cols]
for filename, df in dfs.items():
    for d, c in zip(diff_cols, abs_position_cols):
        df[d] = df[c].diff().shift(-1)
    df[periods_col] = df[time_col].diff().shift(-1)
    df[distances_col] = np.square(df[diff_cols]).sum(axis=1).pow(1./2)
    df[velocity_col] = df[distances_col] / df[periods_col]
    df = df.fillna(method='ffill')


<h3>Visualize results

In [7]:
def show_last_fixation(x, df,target_col='FPOGID', estimated_col='Fixation_ID', pos_cols=['ABS_X', 'ABS_Y']):


    plt.figure(figsize=(18, 10))
    # print target groups
    plt.plot(last_groups_targets[pos_cols[0]], last_groups_targets[pos_cols[1]], 'g-')
    plt.scatter(last_groups_targets[pos_cols[0]], last_groups_targets[pos_cols[1]], marker = 'o' ,s=last_groups_targets['size']**2,c = 'g', alpha=0.4)
    
    # print estimated groups
    
    plt.plot(last_groups_est[pos_cols[0]], last_groups_est[pos_cols[1]], 'r-')
    plt.scatter(last_groups_est[pos_cols[0]], last_groups_est[pos_cols[1]], marker = 'o' ,s=last_groups_est['size']**2,c = 'r', alpha=0.4)
    plt.xlim(-margin, pixel_width + margin)
    plt.ylim(-margin, pixel_height + margin)
    plt.show()
    
    print("Targets\n",last_groups_targets)
    print("Est in target intervals\n",last_groups_est)
#     print(df[:x].groupby([estimated_col])[pos_cols].size().drop([-1], axis=0)[-5:])

In [8]:
def visualize_results(df):

    size = df.shape[0]
    sld = widgets.IntSlider(min=1,max=size,step=1,value=10)
    interact(lambda x :show_last_fixation(x ,df),  x = sld)
#     interact(show_last_fixation(x, df),  x = sld)

In [9]:
def render_result_as_video(queue,df, target_col='FPOGID', estimated_col='Fixation_ID', pos_cols=['ABS_X', 'ABS_Y'], scale = 1):
    size = df.shape[0]
#     size = 500

    for x in range(1,size):

        start = time()
        last_group = df.iloc[x][target_col]
        if last_group > 5:
            x0 = df[df[target_col]==last_group-4].first_valid_index()
        else:
            x0 = 0
        last_groups_targets = df[x0:x].groupby([target_col])[pos_cols].last()
        last_groups_targets['size'] = df[x0:x].groupby([target_col])[pos_cols].size()
    
        try:
            last_groups_est = df[x0:x].groupby([estimated_col])[pos_cols].last().drop([-1], axis=0)
            last_groups_est['size'] = df[x0:x].groupby([estimated_col])[pos_cols].size().drop([-1], axis=0)
        except KeyError:
            last_groups_est = df[x0:x].groupby([estimated_col])[pos_cols].last()
            last_groups_est['size'] = df[x0:x].groupby([estimated_col])[pos_cols].size()
        
        queue.put((last_groups_targets,last_groups_est))
        print("Computed {} frame from {}".format(x, size-1), end = '\r', flush = True)
    queue.put(None)
    print("")

In [10]:
def saver(queue,df_name, alg_name, param_name, general_folder = "result_videos", pos_cols=['ABS_X', 'ABS_Y'], scale = 1):
    result_path = Path(general_folder,alg_name,param_name)
    result_path.mkdir(parents = True, exist_ok = True)
    scaled_w = int(pixel_width/scale)
    scaled_h = int(pixel_height/scale)
    writer = cv2.VideoWriter(str(result_path / (str(df_name)+".avi")), fourcc = cv2.VideoWriter_fourcc(*'DIVX'), fps=30,
                                      frameSize=(scaled_w, scaled_h))
    white_bg = np.full((scaled_h, scaled_w, 3), 255, dtype=np.uint8)
    
    while True:
        if not queue.empty():
            res = queue.get()
            if res is not None :
                last_groups_targets, last_groups_est = res
                frame = white_bg.copy()
                
                # draw target groups (green)

                tar_edges = np.int32(last_groups_targets[pos_cols].to_numpy()/scale)
                cv2.polylines(frame,[tar_edges] , False, green, 2,cv2.LINE_AA)
                for point, r, gid in zip(last_groups_targets[pos_cols].to_numpy(),last_groups_targets['size'].to_numpy(),last_groups_targets.index):
                    cv2.circle(frame, tuple(np.int32(point/scale)), r, green, -1,cv2.LINE_AA)
                    cv2.putText(frame,str(gid),tuple(np.int32(point/scale)), cv2.FONT_HERSHEY_SIMPLEX, 0.75,yellow,2,cv2.LINE_AA)
                
                # draw est groups (red)

                est_edges = np.int32(last_groups_est[pos_cols].to_numpy()/scale)
                cv2.polylines(frame, [est_edges], False, red, 2,cv2.LINE_AA)
                for point, r, gid in zip(last_groups_est[pos_cols].to_numpy(),last_groups_est['size'].to_numpy(),last_groups_est.index ):
                    cv2.circle(frame, tuple(np.int32(point/scale)), r, red, -1,cv2.LINE_AA)
                    cv2.putText(frame,str(gid),tuple(np.int32(point/scale)), cv2.FONT_HERSHEY_SIMPLEX, 0.75,blue,2,cv2.LINE_AA)
                writer.write(frame)
            else:
                break
    writer.release()
    print("Video result for ",df_name, "created!")

<h1>I-VT

In [11]:
def vt_fixation_detector(v, dfs):
    print("V =",v)
    total_size = 0
    total_correct = 0
    for filename, df in dfs.items():
        print(filename)
        size = float(df.shape[0])
        total_size += size
        df.loc[df[velocity_col]<=v,fixation_col] = 1.
        df.loc[df[velocity_col]>v,fixation_col] = 0.
        df.loc[df[fixation_col] ==0., fixation_id_col] = -1
        df.loc[df[fixation_col] ==1., fixation_id_col] = df[fixation_col].diff()[df[fixation_col] ==1.].fillna(1).cumsum()
        df[fixation_id_col].fillna(-1,inplace=True)
        df[fixation_id_col] = df[fixation_id_col].astype(int) 
        print("Tracked number of groups:",df[reading_col].nunique())
        print("Estimated number of groups:",df[fixation_id_col].nunique()-1)
    return dfs
                        

In [12]:
velocities = [10, 100, 500, 1000]
for v in velocities:
    dfs_v = vt_fixation_detector(v, copy.deepcopy(dfs))
    for name, df in dfs_v.items():
        queue = Queue()
        t = Thread(target = saver, args = (queue, Path(name).name.split('.')[0], "I-VT", "V_"+str(v)))
        t.start()
        render_result_as_video(queue, df)
        t.join()

V = 10
data\Alex\Alex_text2_all_gaze_labeled.csv
Tracked number of groups: 254
Estimated number of groups: 680
data\Diana\Diana_text2_all_gaze_labeled.csv
Tracked number of groups: 235
Estimated number of groups: 462
data\Misha\Misha_text2_all_gaze_labeled.csv
Tracked number of groups: 238
Estimated number of groups: 377
data\Polina\Polina_text2_all_gaze_labeled.csv
Tracked number of groups: 189
Estimated number of groups: 386
data\Valik\Valik_text2_all_gaze_labeled.csv
Tracked number of groups: 195
Estimated number of groups: 508
Computed 7336 frame from 7336
Video result for  Alex_text2_all_gaze_labeled created!
Computed 6014 frame from 6014
Video result for  Diana_text2_all_gaze_labeled created!
Computed 7223 frame from 7223
Video result for  Misha_text2_all_gaze_labeled created!
Computed 4721 frame from 4721
Video result for  Polina_text2_all_gaze_labeled created!
Computed 5759 frame from 5759
Video result for  Valik_text2_all_gaze_labeled created!
V = 100
data\Alex\Alex_text2_all_

KeyboardInterrupt: 